In [2]:
import pandas as pd
from fastai.tabular.all import *

# Experiment

In [19]:
df1 = pd.read_csv("./rsqa-indice-qualite-air-2022-2024.csv")
df2 = pd.read_csv("./rsqa-indice-qualite-air-2019-2021.csv")
df = pd.concat([df1, df2], ignore_index=True)

In [28]:
df.isna().sum()

stationId    0
valeur       0
dtype: int64

In [20]:
df['date'] = pd.to_datetime(df['date'])
df['datetime'] = df['date'] + pd.to_timedelta(df['heure'], unit='h')
df.head()

,stationId,polluant,valeur,date,heure,datetime
0,103,O3,15,2022-01-15,3,2022-01-15 03:00:00
1,103,NO2,2,2022-01-15,3,2022-01-15 03:00:00
2,103,PM,12,2022-01-15,3,2022-01-15 03:00:00
3,17,CO,1,2022-02-04,21,2022-02-04 21:00:00
4,17,O3,17,2022-02-04,21,2022-02-04 21:00:00


In [21]:
df = df.sort_values(by='datetime')
df.set_index('datetime', inplace=True)

In [24]:
df.drop(['date', 'heure', 'polluant'], axis=1, inplace=True)
df

,stationId,valeur
datetime,,
2019-01-01 00:00:00,50,36
2019-01-01 00:00:00,55,37
2019-01-01 00:00:00,28,7
2019-01-01 00:00:00,7,3
2019-01-01 00:00:00,80,1
...,...,...
2024-04-09 23:00:00,3,7
2024-04-09 23:00:00,3,1
2024-04-09 23:00:00,3,2


In [27]:
# Feature engineering
df['year'] = df.index.year
df['month'] = df.index.month
df['day'] = df.index.day
df['weekday'] = df.index.weekday
df['hour'] = df.index.hour

Index([2019, 2019, 2019, 2019, 2019, 2019, 2019, 2019, 2019, 2019,
       ...
       2024, 2024, 2024, 2024, 2024, 2024, 2024, 2024, 2024, 2024],
      dtype='int32', name='datetime', length=1789045)

# Run

In [15]:
def process_csv(x):
  x['date'] = pd.to_datetime(df['date'])
  x['datetime'] = df['date'] + pd.to_timedelta(df['heure'], unit='h')
  x.sort_values(by='datetime', inplace=True)
  x.set_index('datetime', inplace=True)
  x.drop(['date', 'heure', 'polluant'], axis=1, inplace=True)
  x['year'] = df.index.year
  x['month'] = df.index.month
  x['day'] = df.index.day
  x['weekday'] = df.index.weekday
  x['hour'] = df.index.hour

def split(x):
  x_train = x[x.index.year <= 2024 & x.index.month <= 6]
  x_test = x[x.index.year >= 2024 & x.index.month > 6]
  return x_train, x_test  

df = pd.read_csv("https://donnees.montreal.ca/dataset/547b8052-1710-4d69-8760-beaa3aa35ec6/resource/0c325562-e742-4e8e-8c36-971f3c9e58cd/download/rsqa-indice-qualite-air-2022-2024.csv")
# df2 = pd.read_csv("https://donnees.montreal.ca/dataset/547b8052-1710-4d69-8760-beaa3aa35ec6/resource/e43dc1d6-fbdd-49c3-a79f-83f63404c281/download/rsqa-indice-qualite-air-2019-2021.csv")
# df = pd.concat([df1, df2], ignore_index=True)

process_csv(df)
df_train, df_test = split(df)


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [12]:
dls = to = TabularPandas(
  df_train,
  procs=[Normalize],
  cat_names=[],
  cont_names=['year', 'month', 'day', 'weekday', 'hour'],
  y_names=['valeur'],
  splits=RandomSplitter(valid_pct=0.2, seed=42)(range_of(df_train))
).dataloaders()

dls.show_batch()

,year,month,day,weekday,hour,valeur
0,2023.0,1.0,6.0,4.0,4.000000e+00,1.0
1,2022.0,7.0,24.0,6.0,1.300000e+01,9.0
2,2021.0,8.0,29.0,6.0,2.100000e+01,1.0
3,2022.0,2.0,10.0,3.0,9.000000e+00,22.0
4,2021.0,7.0,9.0,4.0,9.000000e+00,4.0
5,2021.0,12.0,18.0,5.0,2.200000e+01,1.0
6,2020.0,5.0,6.0,2.0,3.000000e+00,16.0
7,2022.0,1.0,16.0,6.0,4.000000e+00,1.0
8,2020.0,5.0,8.0,4.0,1.800000e+01,1.0
9,2019.0,12.0,10.0,1.0,-3.101038e-07,1.0


In [13]:
learn = tabular_learner(dls, metrics=accuracy)
learn.fit_one_cycle(5)

epoch,train_loss,valid_loss,accuracy,time
0,71.732727,77.100487,0.012763,03:55
1,66.964027,76.102036,0.012763,03:40
2,83.192200,76.630608,0.012763,03:43


KeyboardInterrupt: 